# Notebook 05: Simulación de Ruido y Denoising en CT

Este notebook explora técnicas de simulación de ruido y reducción de ruido (denoising) en imágenes CT pulmonares.

## Contenido:
1. **Simulación de Ruido**: Convertir NDCT (Normal Dose) a LDCT (Low Dose)
2. **Modelos de Ruido**: Poisson
3. **Técnicas de Denoising**: Filtros clásicos vs Deep Learning
4. **Métricas de Calidad**: PSNR, SSIM, MSE
5. **Dataset Mayo Clinic (TCIA)**: Pares NDCT/LDCT simulados con un algoritmo validado

---

## Contexto: NDCT vs LDCT

| Tipo | Dosis | Ruido | Uso Clínico |
|------|-------|-------|-------------|
| **NDCT** (Normal Dose CT) | ~definir | Bajo | Diagnóstico estándar |
| **LDCT** (Low Dose CT) | ~definir | Alto | Screening, seguimiento |

**Objetivo**: 
- Simular ruido LDCT a partir de NDCT para entrenar modelos de denoising
- Usar datos reales del dataset Mayo Clinic LDCT (TCIA) para entrenar modelos de denoising
- Comparar resultado de ambos

---

## Datasets Disponibles

| Dataset | Tipo | Tamaño | Acceso |
|---------|------|--------|--------|
| **LUNA16** | Solo NDCT (simular ruido) | ~13 GB | Automático |
| **Mayo Clinic LDCT (TCIA)** | Pares NDCT/LDCT reales | ~1.32 TB | Manual (requiere descarga) |

**Referencia**: 
- Chen et al. 2016 - "An Open Library of CT Patient Projection Data" (SPIE)
- 
            

---

## 1. Configuración del Entorno

In [ ]:
# Detectar si estamos en Google Colab
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Ejecutando en Google Colab")
    print("="*50)
    
    # Instalar dependencias
    print("\nInstalando dependencias...")
    import subprocess
    paquetes = ['SimpleITK', 'scikit-image', 'requests', 'tqdm']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)
    
    # Clonar repositorio desde GitHub
    print("\nClonando repositorio desde GitHub...")
    repo_url = "https://github.com/Daspony/Imagenes-Biomedicas.git"
    repo_name = "Imagenes-Biomedicas"
    
    if not os.path.exists(f"/content/{repo_name}"):
        subprocess.run(["git", "clone", repo_url], cwd="/content", check=True)
        print(f"Repositorio clonado en /content/{repo_name}")
    else:
        print(f"Repositorio ya existe en /content/{repo_name}")
    
    # Añadir al path
    sys.path.insert(0, f"/content/{repo_name}")
    
    print("Configuración de Colab completada\n")
    
else:
    print("Ejecutando localmente")
    print("="*50)
    
    # Añadir directorio padre al path para importar utils
    parent_dir = os.path.abspath('..')
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)
    
    print(f"Directorio de trabajo: {os.getcwd()}")
    print("Configuración local completada\n")

In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

# Procesamiento de imágenes
from skimage.restoration import denoise_tv_chambolle, denoise_bilateral, denoise_nl_means
from skimage.filters import gaussian, median
from skimage.morphology import disk
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.ndimage import uniform_filter, median_filter

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Importar nuestros módulos
from utils import LUNA16DataLoader, LungPreprocessor, LungVisualizer, download_luna16

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nDispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print("\nLibrerías importadas correctamente")

In [ ]:
# Configuración de rutas
if IN_COLAB:
    luna16_path = '/content/LUNA16'
    download_luna16(subsets=0, include_csv=True, download_dir=luna16_path)
    DATA_PATH = os.path.join(luna16_path, 'subset0')
    ANNOTATIONS_PATH = os.path.join(luna16_path, 'annotations.csv')
else:
    project_root = os.path.abspath('..')
    luna16_path = os.path.join(project_root, 'LUNA16')
    download_luna16(subsets=0, include_csv=True, download_dir=luna16_path)
    DATA_PATH = os.path.join(luna16_path, 'subset0')
    ANNOTATIONS_PATH = os.path.join(luna16_path, 'annotations.csv')

# Verificar rutas
print(f"Datos: {DATA_PATH}")
print(f"Anotaciones: {ANNOTATIONS_PATH}")
print(f"Anotaciones existe: {os.path.exists(ANNOTATIONS_PATH)}")

In [ ]:
# Cargar un escaneo de ejemplo
loader = LUNA16DataLoader(DATA_PATH, ANNOTATIONS_PATH)
preprocessor = LungPreprocessor()

# Obtener primer archivo MHD
mhd_files = list(Path(DATA_PATH).glob("*.mhd"))
print(f"Escaneos disponibles: {len(mhd_files)}")

# Cargar escaneo
sample_path = str(mhd_files[0])
ct_scan, origin, spacing = loader.load_itk_image(sample_path)
print(f"\nVolumen cargado: {ct_scan.shape}")
print(f"Spacing: {spacing}")

# Seleccionar un slice central
slice_idx = ct_scan.shape[0] // 2
original_slice = ct_scan[slice_idx].copy()
print(f"\nSlice seleccionado: {slice_idx}")
print(f"Rango HU: [{original_slice.min():.0f}, {original_slice.max():.0f}]")

---

## 2. Modelos de Ruido en CT

El ruido en imágenes CT tiene características específicas:

1. **Ruido Cuántico (Poisson)**: Dominante en CT, proporcional a √(intensidad)
2. **Ruido Electrónico (Gaussiano)**: Del detector, aditivo
3. **Ruido Estructurado**: Artefactos de reconstrucción

### Modelo simplificado para LDCT:
$$I_{LDCT} = I_{NDCT} + \sigma \cdot \sqrt{I_{NDCT}} \cdot \epsilon$$

donde $\epsilon \sim N(0,1)$ y $\sigma$ controla el nivel de ruido.

In [ ]:
def add_gaussian_noise(image, sigma=25):
    """
    Añade ruido gaussiano aditivo
    
    Args:
        image: Imagen original (valores HU)
        sigma: Desviación estándar del ruido
    
    Returns:
        Imagen con ruido gaussiano
    """
    noise = np.random.normal(0, sigma, image.shape)
    noisy = image + noise
    return noisy.astype(image.dtype)


def add_poisson_noise(image, scale=1.0):
    """
    Añade ruido Poisson (ruido cuántico)
    
    Args:
        image: Imagen original (valores HU)
        scale: Factor de escala para intensidad del ruido
    
    Returns:
        Imagen con ruido Poisson
    """
    # Normalizar a valores positivos para Poisson
    min_val = image.min()
    image_shifted = image - min_val + 1  # Asegurar valores positivos
    
    # Escalar para controlar intensidad del ruido
    image_scaled = image_shifted / scale
    
    # Aplicar ruido Poisson
    noisy = np.random.poisson(image_scaled) * scale
    
    # Restaurar rango original
    noisy = noisy + min_val - 1
    
    return noisy.astype(image.dtype)


def add_ct_realistic_noise(image, dose_ratio=0.25, sigma_base=30):
    """
    Simula ruido realista de CT de baja dosis
    
    Combina:
    - Ruido dependiente de la señal (tipo Poisson)
    - Ruido gaussiano de fondo
    
    Args:
        image: Imagen NDCT original (valores HU)
        dose_ratio: Ratio de dosis (0.25 = 1/4 de la dosis normal)
        sigma_base: Nivel base de ruido gaussiano
    
    Returns:
        Imagen simulando LDCT
    """
    # Factor de ruido inversamente proporcional a la raíz de la dosis
    noise_factor = 1.0 / np.sqrt(dose_ratio)
    
    # Ruido dependiente de la señal (aproximación al ruido cuántico)
    # En regiones más densas (valores HU más altos), el ruido es mayor
    image_normalized = (image - image.min()) / (image.max() - image.min() + 1e-8)
    signal_dependent_noise = np.random.normal(0, 1, image.shape) * np.sqrt(image_normalized + 0.1)
    
    # Ruido gaussiano de fondo
    gaussian_noise = np.random.normal(0, 1, image.shape)
    
    # Combinar ruidos
    sigma = sigma_base * noise_factor
    total_noise = sigma * (0.7 * signal_dependent_noise + 0.3 * gaussian_noise)
    
    noisy = image + total_noise
    return noisy.astype(image.dtype)


print("Funciones de ruido definidas:")
print("  - add_gaussian_noise(): Ruido gaussiano aditivo")
print("  - add_poisson_noise(): Ruido Poisson (cuántico)")
print("  - add_ct_realistic_noise(): Simulación LDCT realista")

In [ ]:
# Aplicar diferentes tipos de ruido
noisy_gaussian = add_gaussian_noise(original_slice, sigma=50)
noisy_poisson = add_poisson_noise(original_slice, scale=100)
noisy_ldct = add_ct_realistic_noise(original_slice, dose_ratio=0.25)

# Normalizar para visualización
def normalize_for_display(img, min_hu=-1000, max_hu=400):
    img_clipped = np.clip(img, min_hu, max_hu)
    return (img_clipped - min_hu) / (max_hu - min_hu)

orig_display = normalize_for_display(original_slice)
gauss_display = normalize_for_display(noisy_gaussian)
poisson_display = normalize_for_display(noisy_poisson)
ldct_display = normalize_for_display(noisy_ldct)

In [ ]:
# Visualizar comparación de tipos de ruido
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Fila 1: Imágenes completas
axes[0, 0].imshow(orig_display, cmap='bone')
axes[0, 0].set_title('Original (NDCT)', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(gauss_display, cmap='bone')
axes[0, 1].set_title('Ruido Gaussiano\n(σ=50)', fontsize=12)
axes[0, 1].axis('off')

axes[0, 2].imshow(poisson_display, cmap='bone')
axes[0, 2].set_title('Ruido Poisson\n(scale=100)', fontsize=12)
axes[0, 2].axis('off')

axes[0, 3].imshow(ldct_display, cmap='bone')
axes[0, 3].set_title('LDCT Simulado\n(dose=25%)', fontsize=12)
axes[0, 3].axis('off')

# Fila 2: Zoom en región de interés
y1, y2, x1, x2 = 200, 300, 200, 300  # Región de zoom

axes[1, 0].imshow(orig_display[y1:y2, x1:x2], cmap='bone')
axes[1, 0].set_title('Zoom Original', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(gauss_display[y1:y2, x1:x2], cmap='bone')
axes[1, 1].set_title('Zoom Gaussiano', fontsize=12)
axes[1, 1].axis('off')

axes[1, 2].imshow(poisson_display[y1:y2, x1:x2], cmap='bone')
axes[1, 2].set_title('Zoom Poisson', fontsize=12)
axes[1, 2].axis('off')

axes[1, 3].imshow(ldct_display[y1:y2, x1:x2], cmap='bone')
axes[1, 3].set_title('Zoom LDCT', fontsize=12)
axes[1, 3].axis('off')

plt.suptitle('Comparación de Modelos de Ruido en CT', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular métricas de calidad
print("Métricas de Calidad de Imagen (respecto al original):")
print("="*60)

# Usar imágenes normalizadas para métricas
metrics = []

for name, noisy in [('Gaussiano', gauss_display), 
                    ('Poisson', poisson_display), 
                    ('LDCT Sim.', ldct_display)]:
    psnr_val = psnr(orig_display, noisy, data_range=1.0)
    ssim_val = ssim(orig_display, noisy, data_range=1.0)
    mse_val = np.mean((orig_display - noisy)**2)
    
    metrics.append({
        'Tipo': name,
        'PSNR (dB)': f'{psnr_val:.2f}',
        'SSIM': f'{ssim_val:.4f}',
        'MSE': f'{mse_val:.6f}'
    })
    
    print(f"\n{name}:")
    print(f"  PSNR: {psnr_val:.2f} dB")
    print(f"  SSIM: {ssim_val:.4f}")
    print(f"  MSE:  {mse_val:.6f}")

# Mostrar tabla
print("\n" + "="*60)
metrics_df = pd.DataFrame(metrics)
print(metrics_df.to_string(index=False))

---

## 3. Variación del Nivel de Ruido

Exploramos cómo diferentes niveles de dosis afectan la calidad de imagen.

In [ ]:
# Simular diferentes niveles de dosis
dose_ratios = [1.0, 0.5, 0.25, 0.125, 0.0625]  # 100%, 50%, 25%, 12.5%, 6.25%
dose_labels = ['100%', '50%', '25%', '12.5%', '6.25%']

fig, axes = plt.subplots(2, 5, figsize=(20, 8))

for i, (dose, label) in enumerate(zip(dose_ratios, dose_labels)):
    if dose == 1.0:
        noisy = original_slice.copy()
    else:
        noisy = add_ct_realistic_noise(original_slice, dose_ratio=dose)
    
    noisy_display = normalize_for_display(noisy)
    
    # Imagen completa
    axes[0, i].imshow(noisy_display, cmap='bone')
    axes[0, i].set_title(f'Dosis: {label}', fontsize=12)
    axes[0, i].axis('off')
    
    # Zoom
    axes[1, i].imshow(noisy_display[200:300, 200:300], cmap='bone')
    
    # Calcular PSNR solo para imágenes con ruido
    if dose < 1.0:
        psnr_val = psnr(orig_display, noisy_display, data_range=1.0)
        axes[1, i].set_title(f'PSNR: {psnr_val:.1f} dB', fontsize=11)
    else:
        axes[1, i].set_title('Referencia', fontsize=11)
    axes[1, i].axis('off')

plt.suptitle('Efecto del Nivel de Dosis en la Calidad de Imagen CT', fontsize=14)
plt.tight_layout()
plt.show()

---

## 4. Técnicas Clásicas de Denoising

Antes de deep learning, se usaban filtros clásicos:

| Técnica | Descripción | Pros | Contras |
|---------|-------------|------|----------|
| Gaussiano | Suavizado isotrópico | Rápido | Pierde bordes |
| Mediana | Filtro no lineal | Preserva bordes | Pierde detalles finos |
| Bilateral | Preserva bordes | Buenos resultados | Lento |
| TV (Total Variation) | Minimiza variación | Preserva bordes | Efecto "cartoon" |
| NLM (Non-Local Means) | Usa parches similares | Muy buenos resultados | Muy lento |

In [ ]:
def apply_classical_denoising(image_normalized):
    """
    Aplica diferentes técnicas clásicas de denoising
    
    Args:
        image_normalized: Imagen normalizada [0, 1]
    
    Returns:
        dict: Diccionario con resultados de cada técnica
    """
    results = {}
    
    # 1. Filtro Gaussiano
    results['Gaussiano'] = gaussian(image_normalized, sigma=1.5)
    
    # 2. Filtro Mediana
    results['Mediana'] = median_filter(image_normalized, size=3)
    
    # 3. Filtro Bilateral
    results['Bilateral'] = denoise_bilateral(
        image_normalized, 
        sigma_color=0.1, 
        sigma_spatial=5,
        channel_axis=None
    )
    
    # 4. Total Variation
    results['TV'] = denoise_tv_chambolle(image_normalized, weight=0.1)
    
    # 5. Non-Local Means (versión rápida)
    results['NLM'] = denoise_nl_means(
        image_normalized,
        h=0.1,
        patch_size=5,
        patch_distance=6,
        channel_axis=None
    )
    
    return results


print("Funciones de denoising clásico definidas")

In [ ]:
# Aplicar denoising a imagen LDCT simulada
print("Aplicando técnicas de denoising...")
print("(Esto puede tomar unos segundos)")

denoised_results = apply_classical_denoising(ldct_display)

print("\nDenoising completado!")

In [ ]:
# Visualizar resultados de denoising
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Fila 1: Imágenes completas
axes[0, 0].imshow(orig_display, cmap='bone')
axes[0, 0].set_title('Original (NDCT)', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(ldct_display, cmap='bone')
psnr_noisy = psnr(orig_display, ldct_display, data_range=1.0)
axes[0, 1].set_title(f'LDCT Simulado\nPSNR: {psnr_noisy:.1f} dB', fontsize=12)
axes[0, 1].axis('off')

# Mejores técnicas
best_techniques = ['Bilateral', 'NLM']
for i, tech in enumerate(best_techniques):
    denoised = denoised_results[tech]
    psnr_val = psnr(orig_display, denoised, data_range=1.0)
    axes[0, i+2].imshow(denoised, cmap='bone')
    axes[0, i+2].set_title(f'{tech}\nPSNR: {psnr_val:.1f} dB', fontsize=12)
    axes[0, i+2].axis('off')

# Fila 2: Zoom
y1, y2, x1, x2 = 200, 300, 200, 300

axes[1, 0].imshow(orig_display[y1:y2, x1:x2], cmap='bone')
axes[1, 0].set_title('Zoom Original', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(ldct_display[y1:y2, x1:x2], cmap='bone')
axes[1, 1].set_title('Zoom LDCT', fontsize=12)
axes[1, 1].axis('off')

for i, tech in enumerate(best_techniques):
    denoised = denoised_results[tech]
    axes[1, i+2].imshow(denoised[y1:y2, x1:x2], cmap='bone')
    axes[1, i+2].set_title(f'Zoom {tech}', fontsize=12)
    axes[1, i+2].axis('off')

plt.suptitle('Comparación de Técnicas de Denoising', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Tabla comparativa de todas las técnicas
print("Comparación de Técnicas de Denoising")
print("="*65)

comparison = []
comparison.append({
    'Técnica': 'LDCT (con ruido)',
    'PSNR (dB)': psnr(orig_display, ldct_display, data_range=1.0),
    'SSIM': ssim(orig_display, ldct_display, data_range=1.0)
})

for tech, denoised in denoised_results.items():
    comparison.append({
        'Técnica': tech,
        'PSNR (dB)': psnr(orig_display, denoised, data_range=1.0),
        'SSIM': ssim(orig_display, denoised, data_range=1.0)
    })

comparison_df = pd.DataFrame(comparison)
comparison_df['PSNR (dB)'] = comparison_df['PSNR (dB)'].round(2)
comparison_df['SSIM'] = comparison_df['SSIM'].round(4)

# Ordenar por PSNR
comparison_df = comparison_df.sort_values('PSNR (dB)', ascending=False)
print(comparison_df.to_string(index=False))

print("\n" + "="*65)
print("Mayor PSNR y SSIM = Mejor calidad de imagen")

---

## 5. Denoising con Deep Learning: DnCNN

**DnCNN** (Denoising Convolutional Neural Network) es una arquitectura clásica para denoising:

- Aprende el **residuo** (ruido) en lugar de la imagen limpia
- Usa batch normalization y ReLU
- Típicamente 17-20 capas convolucionales

$$\hat{x} = y - f(y; \theta)$$

donde $y$ es la imagen ruidosa, $f$ predice el ruido, y $\hat{x}$ es la imagen denoised.

In [ ]:
class DnCNN(nn.Module):
    """
    DnCNN para denoising de imágenes CT
    
    Arquitectura:
    - Conv + ReLU (primera capa)
    - (Conv + BN + ReLU) x (depth-2) capas intermedias
    - Conv (última capa)
    
    El modelo predice el residuo (ruido) que se resta de la entrada.
    """
    def __init__(self, in_channels=1, out_channels=1, num_features=64, depth=17):
        super(DnCNN, self).__init__()
        
        layers = []
        
        # Primera capa: Conv + ReLU
        layers.append(nn.Conv2d(in_channels, num_features, kernel_size=3, padding=1, bias=False))
        layers.append(nn.ReLU(inplace=True))
        
        # Capas intermedias: Conv + BN + ReLU
        for _ in range(depth - 2):
            layers.append(nn.Conv2d(num_features, num_features, kernel_size=3, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(num_features))
            layers.append(nn.ReLU(inplace=True))
        
        # Última capa: Conv (sin activación)
        layers.append(nn.Conv2d(num_features, out_channels, kernel_size=3, padding=1, bias=False))
        
        self.dncnn = nn.Sequential(*layers)
    
    def forward(self, x):
        """Predice el residuo y lo resta de la entrada"""
        residual = self.dncnn(x)
        return x - residual  # Imagen denoised


# Crear modelo
model_dncnn = DnCNN(in_channels=1, out_channels=1, num_features=64, depth=17).to(device)

print("DnCNN creado")
print(f"Parámetros totales: {sum(p.numel() for p in model_dncnn.parameters()):,}")
print(f"Dispositivo: {device}")

In [ ]:
class CTDenoisingDataset(Dataset):
    """
    Dataset para entrenamiento de denoising en CT
    
    Genera pares (imagen_ruidosa, imagen_limpia) a partir de
    escaneos CT aplicando ruido simulado.
    """
    def __init__(self, data_path, annotations_path, patch_size=64, 
                 num_patches_per_scan=50, dose_ratio=0.25):
        self.patch_size = patch_size
        self.num_patches = num_patches_per_scan
        self.dose_ratio = dose_ratio
        
        # Cargar datos
        self.loader = LUNA16DataLoader(data_path, annotations_path)
        self.mhd_files = list(Path(data_path).glob("*.mhd"))
        
        # Cache de slices
        self._slices_cache = []
        self._load_slices()
    
    def _load_slices(self):
        """Carga slices de algunos escaneos"""
        print("Cargando slices para dataset de denoising...")
        
        # Usar solo primeros N escaneos para demo
        max_scans = min(5, len(self.mhd_files))
        
        for mhd_file in tqdm(self.mhd_files[:max_scans]):
            ct_scan, _, _ = self.loader.load_itk_image(str(mhd_file))
            
            # Tomar slices centrales (evitar bordes)
            start_slice = ct_scan.shape[0] // 4
            end_slice = 3 * ct_scan.shape[0] // 4
            
            for slice_idx in range(start_slice, end_slice, 5):  # Cada 5 slices
                ct_slice = ct_scan[slice_idx]
                # Normalizar
                ct_slice = self.loader.normalize_hu(ct_slice, -1000, 400)
                self._slices_cache.append(ct_slice)
        
        print(f"Slices cargados: {len(self._slices_cache)}")
    
    def __len__(self):
        return len(self._slices_cache) * self.num_patches
    
    def _extract_random_patch(self, image):
        """Extrae un patch aleatorio de la imagen"""
        h, w = image.shape
        
        # Asegurar que el patch cabe
        max_y = h - self.patch_size
        max_x = w - self.patch_size
        
        if max_y <= 0 or max_x <= 0:
            # Imagen muy pequeña, redimensionar
            return image[:self.patch_size, :self.patch_size]
        
        y = np.random.randint(0, max_y)
        x = np.random.randint(0, max_x)
        
        return image[y:y+self.patch_size, x:x+self.patch_size]
    
    def __getitem__(self, idx):
        # Seleccionar slice
        slice_idx = idx // self.num_patches
        clean_slice = self._slices_cache[slice_idx]
        
        # Extraer patch aleatorio
        clean_patch = self._extract_random_patch(clean_slice)
        
        # Añadir ruido
        # Convertir a HU aproximado para aplicar ruido realista
        clean_hu = clean_patch * 1400 - 1000  # Desnormalizar
        noisy_hu = add_ct_realistic_noise(clean_hu, dose_ratio=self.dose_ratio)
        noisy_patch = (noisy_hu + 1000) / 1400  # Renormalizar
        noisy_patch = np.clip(noisy_patch, 0, 1)
        
        # Convertir a tensores
        clean_tensor = torch.FloatTensor(clean_patch).unsqueeze(0)
        noisy_tensor = torch.FloatTensor(noisy_patch).unsqueeze(0)
        
        return noisy_tensor, clean_tensor


print("Dataset de denoising definido")

In [ ]:
# Crear dataset y dataloader
denoise_dataset = CTDenoisingDataset(
    DATA_PATH, 
    ANNOTATIONS_PATH,
    patch_size=64,
    num_patches_per_scan=100,
    dose_ratio=0.25
)

denoise_loader = DataLoader(
    denoise_dataset, 
    batch_size=16, 
    shuffle=True,
    num_workers=0
)

print(f"\nDataset size: {len(denoise_dataset)}")
print(f"Batches: {len(denoise_loader)}")

In [ ]:
# Visualizar ejemplos del dataset
noisy_batch, clean_batch = next(iter(denoise_loader))

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i in range(4):
    axes[0, i].imshow(noisy_batch[i, 0].numpy(), cmap='bone')
    axes[0, i].set_title('Ruidosa (LDCT)', fontsize=11)
    axes[0, i].axis('off')
    
    axes[1, i].imshow(clean_batch[i, 0].numpy(), cmap='bone')
    axes[1, i].set_title('Limpia (NDCT)', fontsize=11)
    axes[1, i].axis('off')

plt.suptitle('Ejemplos del Dataset de Denoising (Pares Ruidosa/Limpia)', fontsize=14)
plt.tight_layout()
plt.show()

---

## 6. Entrenamiento del Modelo DnCNN

In [ ]:
def train_denoiser(model, dataloader, epochs=10, lr=1e-3):
    """
    Entrena el modelo de denoising
    
    Args:
        model: Modelo DnCNN
        dataloader: DataLoader con pares (ruidosa, limpia)
        epochs: Número de épocas
        lr: Learning rate
    
    Returns:
        history: Diccionario con historial de entrenamiento
    """
    model.train()
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    history = {'loss': [], 'psnr': []}
    
    for epoch in range(epochs):
        epoch_loss = 0
        epoch_psnr = 0
        num_batches = 0
        
        pbar = tqdm(dataloader, desc=f'Época {epoch+1}/{epochs}')
        
        for noisy, clean in pbar:
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            # Forward
            optimizer.zero_grad()
            denoised = model(noisy)
            loss = criterion(denoised, clean)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            # Métricas
            epoch_loss += loss.item()
            
            # Calcular PSNR
            with torch.no_grad():
                mse = F.mse_loss(denoised, clean)
                batch_psnr = 10 * torch.log10(1.0 / mse)
                epoch_psnr += batch_psnr.item()
            
            num_batches += 1
            pbar.set_postfix({'Loss': f'{loss.item():.6f}', 'PSNR': f'{batch_psnr.item():.2f}'})
        
        scheduler.step()
        
        avg_loss = epoch_loss / num_batches
        avg_psnr = epoch_psnr / num_batches
        
        history['loss'].append(avg_loss)
        history['psnr'].append(avg_psnr)
        
        print(f'Época {epoch+1}: Loss = {avg_loss:.6f}, PSNR = {avg_psnr:.2f} dB')
    
    return history


print("Función de entrenamiento definida")

In [ ]:
# Entrenar el modelo (pocas épocas para demo)
print("Iniciando entrenamiento...")
print("="*60)

history = train_denoiser(model_dncnn, denoise_loader, epochs=5, lr=1e-3)

print("\n" + "="*60)
print("Entrenamiento completado!")

In [ ]:
# Visualizar historial de entrenamiento
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['loss'], 'b-o', linewidth=2, markersize=8)
axes[0].set_xlabel('Época', fontsize=12)
axes[0].set_ylabel('Loss (MSE)', fontsize=12)
axes[0].set_title('Pérdida durante Entrenamiento', fontsize=14)
axes[0].grid(True, alpha=0.3)

# PSNR
axes[1].plot(history['psnr'], 'g-o', linewidth=2, markersize=8)
axes[1].set_xlabel('Época', fontsize=12)
axes[1].set_ylabel('PSNR (dB)', fontsize=12)
axes[1].set_title('PSNR durante Entrenamiento', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Historial de Entrenamiento DnCNN', fontsize=14)
plt.tight_layout()
plt.show()

---

## 7. Evaluación del Modelo

In [ ]:
def evaluate_denoiser(model, image_clean, dose_ratio=0.25):
    """
    Evalúa el modelo de denoising en una imagen
    
    Args:
        model: Modelo entrenado
        image_clean: Imagen limpia normalizada [0, 1]
        dose_ratio: Ratio de dosis para simular LDCT
    
    Returns:
        dict: Resultados con imágenes y métricas
    """
    model.eval()
    
    # Simular LDCT
    clean_hu = image_clean * 1400 - 1000
    noisy_hu = add_ct_realistic_noise(clean_hu, dose_ratio=dose_ratio)
    noisy = (noisy_hu + 1000) / 1400
    noisy = np.clip(noisy, 0, 1)
    
    # Preparar tensor
    noisy_tensor = torch.FloatTensor(noisy).unsqueeze(0).unsqueeze(0).to(device)
    
    # Inferencia
    with torch.no_grad():
        denoised_tensor = model(noisy_tensor)
    
    denoised = denoised_tensor.squeeze().cpu().numpy()
    denoised = np.clip(denoised, 0, 1)
    
    # Calcular métricas
    psnr_noisy = psnr(image_clean, noisy, data_range=1.0)
    psnr_denoised = psnr(image_clean, denoised, data_range=1.0)
    
    ssim_noisy = ssim(image_clean, noisy, data_range=1.0)
    ssim_denoised = ssim(image_clean, denoised, data_range=1.0)
    
    return {
        'clean': image_clean,
        'noisy': noisy,
        'denoised': denoised,
        'psnr_noisy': psnr_noisy,
        'psnr_denoised': psnr_denoised,
        'ssim_noisy': ssim_noisy,
        'ssim_denoised': ssim_denoised,
        'psnr_gain': psnr_denoised - psnr_noisy,
        'ssim_gain': ssim_denoised - ssim_noisy
    }


print("Función de evaluación definida")

In [ ]:
# Evaluar en imagen de prueba
results = evaluate_denoiser(model_dncnn, orig_display, dose_ratio=0.25)

print("Métricas de Evaluación:")
print("="*50)
print(f"\nImagen LDCT (con ruido):")
print(f"  PSNR: {results['psnr_noisy']:.2f} dB")
print(f"  SSIM: {results['ssim_noisy']:.4f}")

print(f"\nImagen Denoised (DnCNN):")
print(f"  PSNR: {results['psnr_denoised']:.2f} dB (+{results['psnr_gain']:.2f} dB)")
print(f"  SSIM: {results['ssim_denoised']:.4f} (+{results['ssim_gain']:.4f})")

In [ ]:
# Visualizar resultados de denoising con DnCNN
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Fila 1: Imágenes completas
axes[0, 0].imshow(results['clean'], cmap='bone')
axes[0, 0].set_title('Original (NDCT)', fontsize=12)
axes[0, 0].axis('off')

axes[0, 1].imshow(results['noisy'], cmap='bone')
axes[0, 1].set_title(f'LDCT Simulado\nPSNR: {results["psnr_noisy"]:.1f} dB', fontsize=12)
axes[0, 1].axis('off')

axes[0, 2].imshow(results['denoised'], cmap='bone')
axes[0, 2].set_title(f'DnCNN Denoised\nPSNR: {results["psnr_denoised"]:.1f} dB', fontsize=12)
axes[0, 2].axis('off')

# Diferencia (residuo aprendido)
residual = np.abs(results['noisy'] - results['denoised'])
axes[0, 3].imshow(residual, cmap='hot')
axes[0, 3].set_title('Ruido Removido\n(Residuo)', fontsize=12)
axes[0, 3].axis('off')

# Fila 2: Zoom
y1, y2, x1, x2 = 200, 300, 200, 300

axes[1, 0].imshow(results['clean'][y1:y2, x1:x2], cmap='bone')
axes[1, 0].set_title('Zoom Original', fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(results['noisy'][y1:y2, x1:x2], cmap='bone')
axes[1, 1].set_title('Zoom LDCT', fontsize=12)
axes[1, 1].axis('off')

axes[1, 2].imshow(results['denoised'][y1:y2, x1:x2], cmap='bone')
axes[1, 2].set_title('Zoom DnCNN', fontsize=12)
axes[1, 2].axis('off')

axes[1, 3].imshow(residual[y1:y2, x1:x2], cmap='hot')
axes[1, 3].set_title('Zoom Residuo', fontsize=12)
axes[1, 3].axis('off')

plt.suptitle('Resultados de Denoising con DnCNN', fontsize=14)
plt.tight_layout()
plt.show()

---

## 8. Comparación: Clásico vs Deep Learning

In [ ]:
# Comparación final de todas las técnicas
print("Comparación Final: Clásico vs Deep Learning")
print("="*65)

# Aplicar denoising clásico a la misma imagen ruidosa
classical_on_same = apply_classical_denoising(results['noisy'])

final_comparison = []

# LDCT (referencia)
final_comparison.append({
    'Técnica': 'LDCT (sin procesar)',
    'Tipo': 'Referencia',
    'PSNR (dB)': results['psnr_noisy'],
    'SSIM': results['ssim_noisy']
})

# Técnicas clásicas
for tech, denoised in classical_on_same.items():
    final_comparison.append({
        'Técnica': tech,
        'Tipo': 'Clásico',
        'PSNR (dB)': psnr(results['clean'], denoised, data_range=1.0),
        'SSIM': ssim(results['clean'], denoised, data_range=1.0)
    })

# DnCNN
final_comparison.append({
    'Técnica': 'DnCNN',
    'Tipo': 'Deep Learning',
    'PSNR (dB)': results['psnr_denoised'],
    'SSIM': results['ssim_denoised']
})

# Crear DataFrame y ordenar
final_df = pd.DataFrame(final_comparison)
final_df['PSNR (dB)'] = final_df['PSNR (dB)'].round(2)
final_df['SSIM'] = final_df['SSIM'].round(4)
final_df = final_df.sort_values('PSNR (dB)', ascending=False)

print(final_df.to_string(index=False))
print("\n" + "="*65)

In [ ]:
# Gráfico de barras comparativo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Ordenar por PSNR para visualización
df_sorted = final_df.sort_values('PSNR (dB)', ascending=True)

# Colores por tipo
colors = ['gray' if t == 'Referencia' else 'steelblue' if t == 'Clásico' else 'coral' 
          for t in df_sorted['Tipo']]

# PSNR
axes[0].barh(df_sorted['Técnica'], df_sorted['PSNR (dB)'], color=colors)
axes[0].set_xlabel('PSNR (dB)', fontsize=12)
axes[0].set_title('Comparación de PSNR', fontsize=14)
axes[0].axvline(x=results['psnr_noisy'], color='red', linestyle='--', alpha=0.5, label='LDCT')
axes[0].grid(True, alpha=0.3, axis='x')

# SSIM
axes[1].barh(df_sorted['Técnica'], df_sorted['SSIM'], color=colors)
axes[1].set_xlabel('SSIM', fontsize=12)
axes[1].set_title('Comparación de SSIM', fontsize=14)
axes[1].axvline(x=results['ssim_noisy'], color='red', linestyle='--', alpha=0.5, label='LDCT')
axes[1].grid(True, alpha=0.3, axis='x')

# Leyenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='gray', label='Referencia'),
    Patch(facecolor='steelblue', label='Clásico'),
    Patch(facecolor='coral', label='Deep Learning')
]
fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))

plt.suptitle('Comparación de Técnicas de Denoising', fontsize=14)
plt.tight_layout()
plt.show()

---

## 9. Guardar Modelo Entrenado

In [ ]:
# Guardar pesos del modelo
if IN_COLAB:
    weights_dir = '/content/weights'
else:
    weights_dir = os.path.join(project_root, 'weights')

os.makedirs(weights_dir, exist_ok=True)

model_path = os.path.join(weights_dir, 'dncnn_denoising.pth')
torch.save(model_dncnn.state_dict(), model_path)

print(f"Modelo guardado en: {model_path}")
print(f"Tamaño: {os.path.getsize(model_path) / 1024 / 1024:.2f} MB")

In [ ]:
# Función para cargar modelo guardado
def load_denoiser(weights_path, device='cuda'):
    """
    Carga un modelo DnCNN entrenado
    
    Args:
        weights_path: Ruta al archivo .pth
        device: Dispositivo ('cuda' o 'cpu')
    
    Returns:
        model: Modelo cargado listo para inferencia
    """
    model = DnCNN(in_channels=1, out_channels=1, num_features=64, depth=17)
    model.load_state_dict(torch.load(weights_path, map_location=device))
    model.to(device)
    model.eval()
    return model


# Verificar que se puede cargar
model_loaded = load_denoiser(model_path, device)
print("Modelo cargado correctamente!")

---

## 10. Dataset Mayo Clinic LDCT (TCIA) - Datos Reales

El **AAPM Low Dose CT Grand Challenge** proporciona pares reales de imágenes NDCT/LDCT del Mayo Clinic.

### Información del Dataset:

| Aspecto | Detalle |
|---------|---------|
| **Nombre** | LDCT-and-Projection-data |
| **Fuente** | The Cancer Imaging Archive (TCIA) |
| **Tamaño** | 1.32 TB (299 casos) |
| **Formato** | DICOM estándar + DICOM-CT-PD (proyecciones) |
| **Dosis** | Full-dose (100%) + Quarter-dose (25%) |
| **Anatomía** | Cabeza, Tórax, Abdomen |
| **DOI** | [10.7937/9npb-2637](https://doi.org/10.7937/9npb-2637) |

### Cómo descargar:

1. Visitar: https://www.cancerimagingarchive.net/collection/ldct-and-projection-data/
2. Descargar el NBIA Data Retriever
3. Seleccionar los casos deseados (Chest para pulmón)
4. Descargar imágenes DICOM

### Estructura esperada:
```
Mayo_LDCT/
├── L001/                    # Paciente 001
│   ├── full_dose/          # Imágenes NDCT (100% dosis)
│   │   └── *.dcm
│   └── quarter_dose/       # Imágenes LDCT (25% dosis)
│       └── *.dcm
├── L002/
...
```

In [ ]:
# ============================================================
# CARGA DE DATOS MAYO CLINIC LDCT (TCIA)
# ============================================================

# Configurar ruta al dataset Mayo Clinic (si está disponible)
if IN_COLAB:
    MAYO_PATH = '/content/Mayo_LDCT'
else:
    MAYO_PATH = os.path.join(project_root, 'Mayo_LDCT')

MAYO_AVAILABLE = os.path.exists(MAYO_PATH)

if MAYO_AVAILABLE:
    print(f"Dataset Mayo Clinic encontrado en: {MAYO_PATH}")
else:
    print(f"Dataset Mayo Clinic NO encontrado en: {MAYO_PATH}")
    print("\nPara usar datos reales NDCT/LDCT:")
    print("1. Descargar de: https://www.cancerimagingarchive.net/collection/ldct-and-projection-data/")
    print("2. Colocar en la carpeta 'Mayo_LDCT' del proyecto")
    print("\nContinuando con ruido simulado...")

In [ ]:
import pydicom
from glob import glob

class MayoLDCTLoader:
    """
    Cargador para el dataset Mayo Clinic LDCT (TCIA)
    
    El dataset contiene pares de imágenes:
    - Full-dose (NDCT): 100% de la dosis de radiación
    - Quarter-dose (LDCT): 25% de la dosis (simulado con ruido Poisson)
    
    Referencia: Chen et al. 2016 - SPIE Medical Imaging
    """
    
    def __init__(self, base_path):
        """
        Args:
            base_path: Ruta base del dataset Mayo_LDCT
        """
        self.base_path = Path(base_path)
        self.patients = self._find_patients()
        
    def _find_patients(self):
        """Encuentra todos los pacientes disponibles"""
        patients = []
        
        # Buscar carpetas de pacientes (formato L001, L002, etc.)
        for patient_dir in sorted(self.base_path.glob("L*")):
            if patient_dir.is_dir():
                patients.append(patient_dir.name)
        
        # También buscar formato alternativo (carpetas con SeriesInstanceUID)
        if len(patients) == 0:
            for patient_dir in sorted(self.base_path.iterdir()):
                if patient_dir.is_dir():
                    patients.append(patient_dir.name)
        
        return patients
    
    def _find_dose_folders(self, patient_id):
        """
        Encuentra las carpetas de full-dose y quarter-dose para un paciente
        
        Returns:
            tuple: (full_dose_path, quarter_dose_path) o (None, None) si no encuentra
        """
        patient_path = self.base_path / patient_id
        
        full_dose = None
        quarter_dose = None
        
        # Buscar por nombre de carpeta
        for folder in patient_path.rglob("*"):
            if folder.is_dir():
                folder_lower = folder.name.lower()
                
                # Detectar full dose
                if any(x in folder_lower for x in ['full', '100', 'nd', 'normal']):
                    full_dose = folder
                # Detectar quarter dose  
                elif any(x in folder_lower for x in ['quarter', '25', 'ld', 'low']):
                    quarter_dose = folder
                # Detectar por contenido de DICOM
                elif not full_dose or not quarter_dose:
                    dcm_files = list(folder.glob("*.dcm"))
                    if dcm_files:
                        try:
                            ds = pydicom.dcmread(str(dcm_files[0]), stop_before_pixels=True)
                            # Intentar detectar por metadatos
                            if hasattr(ds, 'SeriesDescription'):
                                desc = ds.SeriesDescription.lower()
                                if 'full' in desc or '100' in desc:
                                    full_dose = folder
                                elif 'quarter' in desc or '25' in desc:
                                    quarter_dose = folder
                        except:
                            pass
        
        return full_dose, quarter_dose
    
    def load_dicom_series(self, folder_path):
        """
        Carga una serie DICOM completa y la convierte a volumen 3D
        
        Args:
            folder_path: Ruta a la carpeta con archivos .dcm
            
        Returns:
            tuple: (volumen_3d, spacing, origin)
        """
        folder_path = Path(folder_path)
        dcm_files = sorted(folder_path.glob("*.dcm"))
        
        if len(dcm_files) == 0:
            # Buscar en subcarpetas
            dcm_files = sorted(folder_path.rglob("*.dcm"))
        
        if len(dcm_files) == 0:
            raise ValueError(f"No se encontraron archivos DICOM en {folder_path}")
        
        # Leer todos los slices
        slices = []
        for dcm_file in dcm_files:
            ds = pydicom.dcmread(str(dcm_file))
            slices.append(ds)
        
        # Ordenar por posición Z
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        
        # Extraer información espacial
        pixel_spacing = slices[0].PixelSpacing
        slice_thickness = slices[0].SliceThickness if hasattr(slices[0], 'SliceThickness') else 1.0
        spacing = np.array([slice_thickness, pixel_spacing[0], pixel_spacing[1]])
        origin = np.array(slices[0].ImagePositionPatient)
        
        # Construir volumen 3D
        volume = np.stack([s.pixel_array for s in slices])
        
        # Convertir a valores HU
        intercept = slices[0].RescaleIntercept if hasattr(slices[0], 'RescaleIntercept') else 0
        slope = slices[0].RescaleSlope if hasattr(slices[0], 'RescaleSlope') else 1
        volume = volume * slope + intercept
        
        return volume.astype(np.float32), spacing, origin
    
    def load_patient_pair(self, patient_id):
        """
        Carga el par NDCT/LDCT para un paciente
        
        Args:
            patient_id: ID del paciente (ej: 'L001')
            
        Returns:
            dict: {
                'ndct': volumen full-dose,
                'ldct': volumen quarter-dose,
                'spacing': espaciado,
                'patient_id': ID
            }
        """
        full_path, quarter_path = self._find_dose_folders(patient_id)
        
        if full_path is None or quarter_path is None:
            raise ValueError(f"No se encontraron carpetas full/quarter dose para {patient_id}")
        
        print(f"Cargando {patient_id}...")
        print(f"  Full dose: {full_path}")
        print(f"  Quarter dose: {quarter_path}")
        
        ndct, spacing, origin = self.load_dicom_series(full_path)
        ldct, _, _ = self.load_dicom_series(quarter_path)
        
        return {
            'ndct': ndct,
            'ldct': ldct,
            'spacing': spacing,
            'origin': origin,
            'patient_id': patient_id
        }
    
    def get_slice_pair(self, patient_id, slice_idx):
        """
        Obtiene un par de slices NDCT/LDCT
        
        Args:
            patient_id: ID del paciente
            slice_idx: Índice del slice
            
        Returns:
            tuple: (ndct_slice, ldct_slice)
        """
        data = self.load_patient_pair(patient_id)
        return data['ndct'][slice_idx], data['ldct'][slice_idx]


# Crear loader si el dataset está disponible
if MAYO_AVAILABLE:
    mayo_loader = MayoLDCTLoader(MAYO_PATH)
    print(f"\nPacientes encontrados: {len(mayo_loader.patients)}")
    if len(mayo_loader.patients) > 0:
        print(f"Primeros pacientes: {mayo_loader.patients[:5]}")
else:
    mayo_loader = None
    print("\nUsando datos simulados (LUNA16 + ruido sintético)")

In [ ]:
class MayoLDCTDataset(Dataset):
    """
    Dataset PyTorch para pares NDCT/LDCT del Mayo Clinic
    
    Extrae patches de pares reales full-dose/quarter-dose
    para entrenamiento de modelos de denoising.
    """
    
    def __init__(self, mayo_loader, patient_ids=None, patch_size=64, 
                 num_patches_per_volume=200, transform=None):
        """
        Args:
            mayo_loader: Instancia de MayoLDCTLoader
            patient_ids: Lista de IDs de pacientes a usar (None = todos)
            patch_size: Tamaño del patch cuadrado
            num_patches_per_volume: Patches a extraer por volumen
            transform: Transformaciones opcionales
        """
        self.loader = mayo_loader
        self.patch_size = patch_size
        self.num_patches = num_patches_per_volume
        self.transform = transform
        
        # Usar todos los pacientes si no se especifica
        if patient_ids is None:
            self.patient_ids = mayo_loader.patients
        else:
            self.patient_ids = patient_ids
        
        # Cache de datos
        self._cache = {}
        self._load_data()
    
    def _load_data(self):
        """Carga los datos de todos los pacientes"""
        print(f"Cargando {len(self.patient_ids)} pacientes...")
        
        for patient_id in tqdm(self.patient_ids):
            try:
                data = self.loader.load_patient_pair(patient_id)
                self._cache[patient_id] = data
            except Exception as e:
                print(f"Error cargando {patient_id}: {e}")
        
        print(f"Pacientes cargados: {len(self._cache)}")
    
    def __len__(self):
        return len(self._cache) * self.num_patches
    
    def _normalize(self, image, min_hu=-1000, max_hu=400):
        """Normaliza valores HU a [0, 1]"""
        image = np.clip(image, min_hu, max_hu)
        return (image - min_hu) / (max_hu - min_hu)
    
    def _extract_random_patch(self, ndct_vol, ldct_vol):
        """Extrae un patch aleatorio del mismo lugar en ambos volúmenes"""
        z, h, w = ndct_vol.shape
        
        # Seleccionar slice aleatorio (evitar bordes)
        z_idx = np.random.randint(z // 4, 3 * z // 4)
        
        # Seleccionar posición aleatoria
        y = np.random.randint(0, h - self.patch_size)
        x = np.random.randint(0, w - self.patch_size)
        
        # Extraer patches
        ndct_patch = ndct_vol[z_idx, y:y+self.patch_size, x:x+self.patch_size]
        ldct_patch = ldct_vol[z_idx, y:y+self.patch_size, x:x+self.patch_size]
        
        return ndct_patch, ldct_patch
    
    def __getitem__(self, idx):
        # Seleccionar paciente
        patient_idx = idx // self.num_patches
        patient_id = list(self._cache.keys())[patient_idx]
        data = self._cache[patient_id]
        
        # Extraer patch
        ndct_patch, ldct_patch = self._extract_random_patch(
            data['ndct'], data['ldct']
        )
        
        # Normalizar
        ndct_patch = self._normalize(ndct_patch)
        ldct_patch = self._normalize(ldct_patch)
        
        # Convertir a tensores
        ndct_tensor = torch.FloatTensor(ndct_patch).unsqueeze(0)
        ldct_tensor = torch.FloatTensor(ldct_patch).unsqueeze(0)
        
        # Aplicar transformaciones
        if self.transform:
            ndct_tensor = self.transform(ndct_tensor)
            ldct_tensor = self.transform(ldct_tensor)
        
        # Retornar (ruidosa, limpia) para mantener consistencia con el dataset simulado
        return ldct_tensor, ndct_tensor


# Crear dataset si Mayo está disponible
if MAYO_AVAILABLE and mayo_loader is not None and len(mayo_loader.patients) > 0:
    print("Creando dataset con datos reales Mayo Clinic...")
    
    # Usar primeros 5 pacientes para demo
    demo_patients = mayo_loader.patients[:5]
    mayo_dataset = MayoLDCTDataset(
        mayo_loader, 
        patient_ids=demo_patients,
        patch_size=64,
        num_patches_per_volume=100
    )
    
    mayo_dataloader = DataLoader(
        mayo_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=0
    )
    
    print(f"\nDataset Mayo creado:")
    print(f"  Pacientes: {len(demo_patients)}")
    print(f"  Total patches: {len(mayo_dataset)}")
else:
    mayo_dataset = None
    mayo_dataloader = None
    print("\nDataset Mayo no disponible, usar datos simulados")

In [ ]:
# Visualizar ejemplos de datos reales (si están disponibles)
if mayo_dataloader is not None:
    print("Visualizando pares reales NDCT/LDCT del Mayo Clinic:")
    
    ldct_batch, ndct_batch = next(iter(mayo_dataloader))
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    for i in range(4):
        # LDCT (ruidosa)
        axes[0, i].imshow(ldct_batch[i, 0].numpy(), cmap='bone')
        axes[0, i].set_title('LDCT Real (25% dosis)', fontsize=11)
        axes[0, i].axis('off')
        
        # NDCT (limpia)
        axes[1, i].imshow(ndct_batch[i, 0].numpy(), cmap='bone')
        axes[1, i].set_title('NDCT Real (100% dosis)', fontsize=11)
        axes[1, i].axis('off')
    
    plt.suptitle('Pares Reales NDCT/LDCT - Mayo Clinic Dataset', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Calcular diferencia de ruido real
    diff = np.abs(ldct_batch[0, 0].numpy() - ndct_batch[0, 0].numpy())
    psnr_real = psnr(ndct_batch[0, 0].numpy(), ldct_batch[0, 0].numpy(), data_range=1.0)
    
    print(f"\nMétricas de ruido real (ejemplo):")
    print(f"  PSNR LDCT vs NDCT: {psnr_real:.2f} dB")
    print(f"  Diferencia media: {diff.mean():.4f}")
    print(f"  Diferencia máx: {diff.max():.4f}")
else:
    print("Dataset Mayo no disponible - usando datos simulados")

---

## 11. Entrenamiento con Datos Reales Mayo Clinic

Si el dataset Mayo está disponible, podemos entrenar el modelo DnCNN con pares reales NDCT/LDCT.

In [ ]:
# Entrenar con datos reales Mayo (si están disponibles)
if mayo_dataloader is not None:
    print("="*60)
    print("ENTRENAMIENTO CON DATOS REALES MAYO CLINIC")
    print("="*60)
    
    # Crear nuevo modelo para datos reales
    model_mayo = DnCNN(in_channels=1, out_channels=1, num_features=64, depth=17).to(device)
    
    print(f"\nModelo DnCNN para Mayo Clinic:")
    print(f"  Parámetros: {sum(p.numel() for p in model_mayo.parameters()):,}")
    
    # Entrenar
    print("\nIniciando entrenamiento con datos reales...")
    history_mayo = train_denoiser(model_mayo, mayo_dataloader, epochs=10, lr=1e-3)
    
    # Guardar modelo
    mayo_model_path = os.path.join(weights_dir, 'dncnn_mayo_real.pth')
    torch.save(model_mayo.state_dict(), mayo_model_path)
    print(f"\nModelo Mayo guardado en: {mayo_model_path}")
    
else:
    print("Dataset Mayo no disponible")
    print("El modelo fue entrenado con ruido simulado (ver secciones anteriores)")
    print("\nPara entrenar con datos reales:")
    print("1. Descargar de: https://www.cancerimagingarchive.net/collection/ldct-and-projection-data/")
    print("2. Colocar en carpeta 'Mayo_LDCT'")
    print("3. Re-ejecutar este notebook")

---

## 12. Resumen y Referencias

### Logros de este notebook:

1. **Simulación de ruido LDCT** a partir de imágenes NDCT (Gaussiano, Poisson, CT realista)
2. **Comparación de técnicas clásicas** de denoising (Bilateral, TV, NLM)
3. **Implementación de DnCNN** para denoising con deep learning
4. **Soporte para dataset Mayo Clinic** con pares NDCT/LDCT reales

### Modelos guardados:

| Modelo | Datos | Archivo |
|--------|-------|---------|
| DnCNN (simulado) | LUNA16 + ruido sintético | `weights/dncnn_denoising.pth` |
| DnCNN (real) | Mayo Clinic LDCT | `weights/dncnn_mayo_real.pth` |

### Referencias:

1. **Paper del dataset Mayo Clinic**:
   - Chen B. et al. (2016). "An Open Library of CT Patient Projection Data". SPIE Medical Imaging.
   - DOI: [10.1117/12.2216823](https://doi.org/10.1117/12.2216823)

2. **Dataset TCIA**:
   - LDCT-and-Projection-data
   - DOI: [10.7937/9npb-2637](https://doi.org/10.7937/9npb-2637)
   - URL: https://www.cancerimagingarchive.net/collection/ldct-and-projection-data/

3. **DnCNN**:
   - Zhang K. et al. (2017). "Beyond a Gaussian Denoiser: Residual Learning of Deep CNN for Image Denoising". IEEE TIP.

4. **AAPM Low Dose CT Grand Challenge**:
   - https://www.aapm.org/grandchallenge/lowdosect/

In [ ]:
print("="*60)
print("RESUMEN DEL NOTEBOOK 05: DENOISING")
print("="*60)

print("\n1. MODELOS DE RUIDO:")
print("   - Gaussiano (aditivo)")
print("   - Poisson (cuántico)")
print("   - LDCT realista (combinado)")

print("\n2. TÉCNICAS DE DENOISING:")
print("   Clásicas: Gaussiano, Mediana, Bilateral, TV, NLM")
print("   Deep Learning: DnCNN (17 capas, residual learning)")

print("\n3. DATASETS SOPORTADOS:")
print("   - LUNA16: Ruido simulado (disponible)")
if MAYO_AVAILABLE:
    print(f"   - Mayo Clinic: Pares reales NDCT/LDCT (disponible, {len(mayo_loader.patients)} pacientes)")
else:
    print("   - Mayo Clinic: No descargado (ver instrucciones arriba)")

print("\n4. RESULTADOS (ruido simulado):")
print(f"   LDCT sin procesar: {results['psnr_noisy']:.2f} dB")
print(f"   DnCNN:             {results['psnr_denoised']:.2f} dB (+{results['psnr_gain']:.2f} dB)")

print("\n5. ARCHIVOS GENERADOS:")
print(f"   - weights/dncnn_denoising.pth (ruido simulado)")
if MAYO_AVAILABLE:
    print(f"   - weights/dncnn_mayo_real.pth (datos reales)")

print("\n" + "="*60)
print("Para mejorar resultados:")
print("  1. Descargar Mayo Clinic LDCT de TCIA")
print("  2. Entrenar más épocas (50-100)")
print("  3. Usar arquitecturas avanzadas (RED-CNN, WGAN)")
print("="*60)

In [ ]:
print("="*60)
print("RESUMEN DEL NOTEBOOK 05: DENOISING")
print("="*60)

print("\nModelos de ruido implementados:")
print("  - Gaussiano (aditivo)")
print("  - Poisson (cuántico)")
print("  - LDCT realista (combinado)")

print("\nTécnicas de denoising:")
print("  Clásicas: Gaussiano, Mediana, Bilateral, TV, NLM")
print("  Deep Learning: DnCNN")

print("\nResultados (PSNR en imagen de prueba):")
print(f"  LDCT sin procesar: {results['psnr_noisy']:.2f} dB")
print(f"  DnCNN:             {results['psnr_denoised']:.2f} dB (+{results['psnr_gain']:.2f} dB)")

if model_dncnn is not None:
    print(f"\nModelo DnCNN:")
    print(f"  Parámetros: {sum(p.numel() for p in model_dncnn.parameters()):,}")
    print(f"  Guardado en: {model_path}")

print("\n" + "="*60)